# TMDV MOVIES RECOMMENDATION SYSTEM USING Machine Learnig by Souvik Sarkar
---------------------------------------------------------------------------------
# *Core Theory :*
 'Content Based' Recommendation system using tags (bags of words) ,now user input can be feched and using 'Cosine Similarity' which will provide 5 most familier movies from 5000 movies list

## Installing the libraries needed for the data interpretation

In [ ]:
import numpy as np
import pandas as pd
movie = pd.read_csv('tmdb_5000_movies.csv')
cred = pd.read_csv('tmdb_5000_credits.csv')


## Merging Both with the basis of title

In [ ]:
movies = movie.merge(cred, on='title')


In [ ]:
movies.head(1)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [ ]:
movies.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

### Assumption :
# deleting events:
 budget doesnt always consider for recom. , reg language is very less compared to eng. , runtime is not consider on good and bad , also revenue is not considere for our model
 # colmns based on which pre processed
 movie_id for title pic generator
 **

In [ ]:
mv = movies[['movie_id','title','overview','genres','keywords','cast','crew']]
mv .head(1)

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


## data preprocessing

In [ ]:


mv.isnull().sum()


movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [ ]:
mv.dropna(inplace = True) ## drop the missing data
mv.duplicated().sum()

C:\Users\sanku\AppData\Local\Temp\ipykernel_6584\1052643227.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mv.dropna(inplace = True) ## drop the missing data


np.int64(0)

 correcting the colmns

In [ ]:
mv.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

# importing ast module to convert strings of list to list only
so that Id code will vanishes 

In [ ]:
import ast


In [ ]:
def convert(obj) :
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])      
    return L

In [ ]:

mv['genres'] = mv['genres'].apply(convert)
mv['keywords'] = mv['keywords'].apply(convert)

C:\Users\sanku\AppData\Local\Temp\ipykernel_6584\3292975914.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mv['genres'] = mv['genres'].apply(convert)
C:\Users\sanku\AppData\Local\Temp\ipykernel_6584\3292975914.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mv['keywords'] = mv['keywords'].apply(convert)


In [ ]:
## chosing tp 3 actors from the cast list
def convert_cast( obj) :
    L = []
    f=0
    for i in ast.literal_eval(obj):
        if f != 3: 
            L.append(i['name'])
            f = f+1
        else: 
            break
    return L

In [ ]:
mv['cast']=mv['cast'].apply(convert_cast)

C:\Users\sanku\AppData\Local\Temp\ipykernel_6584\1344718569.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mv['cast']=mv['cast'].apply(convert_cast)


In [ ]:
# Fetching the director name
def director(obj) :
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])  
            break
    return L

In [ ]:
mv['crew']=mv['crew'].apply(director)


C:\Users\sanku\AppData\Local\Temp\ipykernel_6584\3294513638.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mv['crew']=mv['crew'].apply(director)


In [ ]:
# lets see the o/p
mv.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


Applying a function to eliminate the spaces bw names so that our model fits the currect choice by its name


In [ ]:
mv['genres']= mv['genres'].apply( lambda x:[i.replace(" ","") for i  in x])
mv['keywords']= mv['keywords'].apply( lambda x:[i.replace(" ","") for i  in x])
mv['cast']= mv['cast'].apply( lambda x:[i.replace(" ","") for i  in x])
mv['crew']= mv['crew'].apply( lambda x:[i.replace(" ","") for i  in x])

mv.head()

C:\Users\sanku\AppData\Local\Temp\ipykernel_6584\16596423.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mv['genres']= mv['genres'].apply( lambda x:[i.replace(" ","") for i  in x])
C:\Users\sanku\AppData\Local\Temp\ipykernel_6584\16596423.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mv['keywords']= mv['keywords'].apply( lambda x:[i.replace(" ","") for i  in x])
C:\Users\sanku\AppData\Local\Temp\ipykernel_6584\16596423.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [ ]:
# also make overview colmn as list
mv['overview'] = mv['overview'].apply(lambda x: x.split())

C:\Users\sanku\AppData\Local\Temp\ipykernel_6584\50419917.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mv['overview'] = mv['overview'].apply(lambda x: x.split())


In [ ]:
# new tag colm is added to combine all the  other colmns
mv['tags'] = mv['overview'] + mv['genres'] + mv['keywords'] + mv['cast'] + mv['crew']

C:\Users\sanku\AppData\Local\Temp\ipykernel_6584\2338705256.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mv['tags'] = mv['overview'] + mv['genres'] + mv['keywords'] + mv['cast'] + mv['crew']


In [ ]:
# making a new data frame with title ,id, and tags
df = mv[['movie_id','title','tags']]
# making all list in an array
df['tags']= df['tags'].apply(lambda x: " ".join(x))
df['tags']= df['tags'].apply(lambda x: x.lower())

C:\Users\sanku\AppData\Local\Temp\ipykernel_6584\2922572454.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags']= df['tags'].apply(lambda x: " ".join(x))
C:\Users\sanku\AppData\Local\Temp\ipykernel_6584\2922572454.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags']= df['tags'].apply(lambda x: x.lower())


In [ ]:

#see the o/p before further processing
df.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


## TEXT TO VECTOR TRANSFORMATION is the CORE IDEA
dont considering the stop words (are from is he the etc,)
use stemming to get only unque similar words


In [ ]:

pip install nltk


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
df['tags'] = df['tags'].apply(stem)

C:\Users\sanku\AppData\Local\Temp\ipykernel_6584\866399325.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(stem)


In [ ]:
# using this class 
from sklearn.feature_extraction.text import CountVectorizer
cnt_vect = CountVectorizer(max_features = 5000, stop_words='english')

In [ ]:
# run this cell before cell-38
import sklearn


In [ ]:
#cnt_vect.get_feature_names_out()
#ps.stem('dancing')

In [ ]:
# making bag of words
vect = cnt_vect.fit_transform(df['tags']).toarray()
vect 

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], shape=(4806, 5000))

In [ ]:
vect.shape


(4806, 5000)

# now all 4806 movies are taken in 2000 dimentional vectors in space
# we use cosine distance for calculating the distance
# to sort using enumerate for fetching the movies name

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
siml =  cosine_similarity(vect)

In [ ]:
## function to fecth movie 
def recom (movie):
    movie_index = df[df['title'] == movie].index[0]  # 0 th position gives the index no. of the movie
    dists = siml[movie_index]
    movies_list = sorted(list(enumerate(dists)),reverse = True, key = lambda x: x[1])[1:6] ## main logic behind the whole code
    for i in movies_list:
        print(df.iloc[i[0]].title)
    return

In [ ]:
#test 1
recom('The Dark Knight')

The Dark Knight Rises
Batman Begins
Batman Returns
Batman Forever
Batman


In [ ]:
#test2
recom('Life of Pi')

Sahara
Silver Medalist
Deep Rising
Crouching Tiger, Hidden Dragon
Two Brothers


# to save or loading the model pickle lib is used 

In [ ]:
import pickle
# firstly the dataframe from pandas is not working in the web app for some reason so I convert it to dict type file then to another data frame
pickle.dump(df.to_dict(),open('MovieRecomSYS_dict.pkl','wb'))

In [ ]:
import pickle
pickle.dump(siml,open('siml.pkl','wb'))